In [1]:
from webscrapping.database.sql_queries import ValorantQueries
import pandas as pd

In [2]:
vq = ValorantQueries()
vq.set_match(43621)

Connected to database valorant successfully
[('teams',), ('roundevents',), ('players',), ('events',), ('series',), ('matches',), ('maps',), ('rounds',), ('roundeconomies',), ('roundlocations',), ('playermapinstance',), ('assists',)]


In [3]:
states = vq.get_initial_state()
events = vq.get_event_table()
events

,Round,EventIndex,Round_time_millis,PlayerID,PlayerName,VictimID,VictimName,EventType,DamageType,WeaponID,...,Alive_Bazzi,Alive_Fisker,Alive_Munchkin,Alive_Saadhak,Alive_Sacy,Alive_ade,Alive_frz,Alive_gtnziN,Alive_neth,Alive_sutecas
0,1,1,17272,194,sutecas,1824,Fisker,kill,weapon,12,...,1,0,1,1,1,1,1,1,1,1
1,1,2,29339,2895,Saadhak,876,Munchkin,kill,weapon,11,...,1,0,0,1,1,1,1,1,1,1
2,1,3,29760,1877,frz,2388,neth,kill,weapon,12,...,1,0,0,1,1,1,1,1,0,1
3,1,4,37694,2895,Saadhak,0,NaN,plant,None,0,...,1,0,0,1,1,1,1,1,0,1
4,1,5,38461,1877,frz,1075,Bazzi,kill,weapon,12,...,0,0,0,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22,6,93788,2388,neth,0,NaN,plant,None,0,...,0,0,0,1,1,1,0,1,1,0
197,22,7,94458,2294,Sacy,1843,ade,kill,weapon,6,...,0,0,0,1,1,0,0,1,1,0
198,22,8,107910,2388,neth,2895,Saadhak,kill,weapon,19,...,0,0,0,0,1,0,0,1,1,0
199,22,9,109289,2294,Sacy,2388,neth,kill,ability,0,...,0,0,0,0,1,0,0,1,0,0


In [19]:
id_dict = dict(zip(states["Player ID"].unique(), states["Player Name"].unique()))
chosen_round = 5
chosen_millis = 80351
sides = vq.get_player_sides_by_round(5)
attackers = [id_dict[item] for item in sides["attackers"]]
defenders = [id_dict[item] for item in sides["defenders"]]
players = [sub for sub in list(set(attackers + defenders))]
all_players = ["Alive_" + sub for sub in list(set(attackers + defenders))]
attackers_query = states[(states["Round"] == chosen_round) & (states["Player Name"].isin(attackers))]
defenders_query = states[(states["Round"] == chosen_round) & (states["Player Name"].isin(defenders))]
events_query = events[(events["Round"] == chosen_round) & (events["Round_time_millis"] == chosen_millis)]
alive_dict = dict(zip(players, events_query[all_players].iloc[0]))
alive_dict

{'frz': 1,
 'Saadhak': 0,
 'neth': 0,
 'gtnziN': 1,
 'Bazzi': 1,
 'Sacy': 0,
 'sutecas': 1,
 'Fisker': 0,
 'ade': 1,
 'Munchkin': 1}